In [ ]:
import utils
import importlib

from automaton import State, Automaton

importlib.reload(utils)

In [ ]:
infix = '(a+b)*(a+c)+a+b*c'
postfix = utils.infix2postfix(infix)
root = utils.postfix2tree(postfix)
print(root.get_postfix())

root.show_dot()

In [ ]:
def token_type(character):
    if character in ['.', '|']: return 'binary'
    elif character == '*': return 'unary'
    return False

infix = '(a|b)*abb'
infix = utils.add_explicit_concatenation(infix)
postfix = utils.infix2postfix(infix, 
                              priority={'*': 2, '.': 1, '|': 0})
root = utils.postfix2tree(postfix, token_type)
print(infix)
print(postfix)
print(root.get_postfix())

root.show_dot()

In [ ]:
_STATE_REGISTER = []

def create_expression(letter):
    """NFA of a new expression."""
    initial_state = State('s{}'.format(len(_STATE_REGISTER)))
    _STATE_REGISTER.append(initial_state)

    out_state = State('s{}'.format(len(_STATE_REGISTER)))
    _STATE_REGISTER.append(out_state)

    initial_state.add_transition(letter, out_state)
    
    return {'initial_state': initial_state, 
            'out_state': out_state}

def expression_union(expr1, expr2):
    """NFA of the union of two expressions."""
    initial_state = State('s{}'.format(len(_STATE_REGISTER)))
    _STATE_REGISTER.append(initial_state)

    out_state = State('s{}'.format(len(_STATE_REGISTER)))
    _STATE_REGISTER.append(out_state)
    
    initial_state.add_transition('$', [expr1['initial_state'], 
                                       expr2['initial_state']])

    expr1['out_state'].add_transition('$', out_state)
    expr2['out_state'].add_transition('$', out_state)

    return {'initial_state': initial_state, 
            'out_state': out_state} 
    
def expression_star(expr1):
    """NFA of the star of an expression."""
    initial_state = State('s{}'.format(len(_STATE_REGISTER)))
    _STATE_REGISTER.append(initial_state)

    out_state = State('s{}'.format(len(_STATE_REGISTER)))
    _STATE_REGISTER.append(out_state)
    
    initial_state.add_transition('$', [expr1['initial_state'], 
                                       out_state])

    expr1['out_state'].add_transition('$', [expr1['initial_state'], 
                                            out_state])

    return {'initial_state': initial_state, 
            'out_state': out_state} 
    
def expression_concat(expr1, letter):
    """NFA of the concatenation expr1 and the givel letter.
    
    This is more convenient that using create_expression(letter)
    to create an expression corresponding to the letter
    """    
    out_state = State('s{}'.format(len(_STATE_REGISTER)))
    _STATE_REGISTER.append(out_state)
    
    expr1['out_state'].add_transition(letter, out_state)
    
    return {'initial_state': expr1['initial_state'], 
            'out_state': out_state} 
    
expr1 = create_expression('a')
expr2 = create_expression('b')
expr3 = expression_union(expr1, expr2)
expr4 = expression_star(expr3)
expr5 = expression_concat(expr4, 'a')
expr6 = expression_concat(expr5, 'b')
expr7 = expression_concat(expr6, 'b')

M = Automaton('M', _STATE_REGISTER, ['a', 'b'], expr7['initial_state'], expr7['out_state'])

In [ ]:
M.show_dot()